In [ ]:
#H06T05
#aikasarjojen yhdistäminen

In [1]:
def tempdiff(url1, url2):
    import pandas as pd
    from datetime import timedelta

    #ensimmäinen datasetti
    df1 = pd.read_csv(url1, sep=',', parse_dates=["Unnamed: 0"], dayfirst=False)
    df1 = pd.DataFrame(df1[['Unnamed: 0','Air temperature, degC']])
    df1.rename(columns={'Unnamed: 0':'timestamp_1', 'Air temperature, degC': 'Temp_1, C-deg'}, inplace=True)
    df1.set_index(keys='timestamp_1', inplace=True)
    df1.sort_index(ascending=True, inplace=False)
    
    #toinen datasetti on hiukan erilainen
    df2 = pd.read_csv(url2, sep='\t')
    alku = pd.to_datetime('2020-11-05 08:40:00') #asetetaan alkuhetki
    df2['timestamp_2'] = alku + pd.to_timedelta(df2['time, s'], unit='s') #ja lasketaan aika siihen päälle
    df2.rename(columns={'Temp, C-deg':'Temp_2, C-deg'}, inplace=True)
    df2 = df2[['timestamp_2','Temp_2, C-deg']]
    df2.set_index(keys='timestamp_2', inplace=True)
    df2.sort_index(ascending=True, inplace=False)
    
    #yhdistetään joinimalla
    df = df1.join(other=df2, how='outer', sort=True)
    
    #täytetään nanit ja lasketaan...
    df['Temp_1, C-deg'] = df['Temp_1, C-deg'].fillna(method='ffill')
    df['Temp_2, C-deg'] = df['Temp_2, C-deg'].fillna(method='ffill')
    
    #...erotus ja liukuva keskiarvo
    df['Temp_diff, C-deg'] = df['Temp_2, C-deg'] - df['Temp_1, C-deg']
    df['Temp_roll, C-deg'] = df['Temp_diff, C-deg'].rolling(window='3h').mean()
    
    #lopuksi haluttu muotoilu
    df = df.applymap('{:.3f}'.format)
    
    #ja pyydetty tuloste
    tulos = df.loc['2020-11-05 12:59:00':,['Temp_diff, C-deg','Temp_roll, C-deg']].head(5)
    
    return tulos

In [2]:
url1 = "https://student.labranet.jamk.fi/~huoptu/datananal_s20/kerta6/harjoitukset/saatiedot_kumpula_1d.csv"
url2 = "https://student.labranet.jamk.fi/~huoptu/datananal_s20/kerta6/harjoitukset/indoormeas.csv"

print(tempdiff(url1, url2))

                    Temp_diff, C-deg Temp_roll, C-deg
2020-11-05 13:00:00           12.590           12.623
2020-11-05 13:01:00           12.590           12.620
2020-11-05 13:02:00           12.590           12.617
2020-11-05 13:03:00           12.590           12.614
2020-11-05 13:03:59           12.590           12.611
